In [ ]:
!python -m pip install -q wandb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 9.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.5/188.5 kB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 214.7/214.7 kB 5.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 2.5 MB/s eta 0:00:00


In [ ]:
# import necessary packages
import numpy as np
import pandas as pd
import tensorflow as tf
import re

import wandb
from wandb.keras import WandbCallback
from wandb.keras import WandbMetricsLogger
import wandb.apis.reports as wr

wandb: Thanks for trying out the Report API!
wandb: For a tutorial, check out https://colab.research.google.com/drive/1CzyJx1nuOS4pdkXa2XPaRQyZdmFmLmXV
wandb: 
wandb: Try out tab completion to see what's available.
wandb:   ∟ everything:    `wr.<tab>`
wandb:       ∟ panels:    `wr.panels.<tab>`
wandb:       ∟ blocks:    `wr.blocks.<tab>`
wandb:       ∟ helpers:   `wr.helpers.<tab>`
wandb:       ∟ templates: `wr.templates.<tab>`
wandb:       
wandb: For bugs/feature requests, please create an issue on github: https://github.com/wandb/wandb/issues


In [ ]:
# function to count sequence length
def count_seq_length(df: pd.DataFrame, seq_col: str) -> pd.Series:
    pattern = re.compile(r"\[UNIMOD:.*\]", re.IGNORECASE)
    df[seq_col].replace(pattern, "", inplace= True)
    return df[seq_col].str.len()

In [ ]:
TRAIN_DATAPATH = 'https://raw.githubusercontent.com/goldjunge3010/masterpraktikum/main/third_pool_tresh_1_0_train.csv'
train_set = pd.read_csv(TRAIN_DATAPATH)
counts = count_seq_length(pd.read_csv(TRAIN_DATAPATH), "sequence")

In [ ]:
# Initialize WANDB
PROJECT = 'Custom_Vega_Plots'
RUN = "t_run_4"
wandb.init(project = PROJECT, name = RUN)

In [ ]:
def plot_hist(data, type: str):
  """Recieves data and logs a custom histogram to wandb

  Parameters
  ----------
  data : pd.DataFrame, required
      DataFrame or Series that contains the data to be plotted; Dimension: 1 x N
  type : str, optional
      A flag used to choose matching Vega - template
      pep_len: Template for plotting histogram of peptide lengths
      ret_time: Template for plotting histogram of indexed retention times
      None: Template for default histogram

  Returns
  -------
  """
  # log table to wandb
  data.columns = ["x"]
  table = wandb.Table(dataframe = data)

  # extract column name of first column
  field = list(train_set.columns.values)[0]

  print(field)
  # choose matching Vega- template
  match type:
    case "pep_len":
      hist = wandb.plot_table(
      vega_spec_name="master_praktikum/hist_pep_len",
      data_table = table,
      fields = field)
    case "ret_time":
      hist = wandb.plot_table(
      vega_spec_name="master_praktikum/hist_time",
      data_table = table,
      fields = field)
    case _:
      hist = wandb.plot_table(
      vega_spec_name="master_praktikum/def_hist",
      data_table = table,
      fields = field)

  # log histogram to wandb
  wandb.log({"Histogram": hist})


In [ ]:
plot_hist(counts.to_frame(), type = "pep_len")

sequence


In [ ]:
wandb.finish()

In [ ]:
c = counts.to_list()
data = [[s] for s in c]

train_set_table = wandb.Table(dataframe = train_set)
count_table = wandb.Table(data=data, columns=["counts"])


hist = wandb.plot_table(
    vega_spec_name="master_praktikum/hist_pep_len",
    data_table = count_table,
    fields = "counts")

hist2 = wandb.plot_table(
    vega_spec_name="master_praktikum/hist_ret_time",
    data_table = train_set_table,
    fields = "irt")
wandb.log({'pep_len': hist,
           'ret_time': hist2
           })

wandb.finish()

# i dont think its possible to use CustomChart with a vega_spec

In [ ]:
c = counts.to_list()
data = [[s] for s in c]
table = wandb.Table(data=data, columns=["counts"])
histogram = wandb.plot.histogram(table, value='height', title='Histogram')
wandb.log({"hist_1" : histogram})
wandb.finish()